# mapping-challenge-mask_rcnn-training
![CrowdAI-Logo](https://github.com/crowdAI/crowdai/raw/master/app/assets/images/misc/crowdai-logo-smile.svg?sanitize=true)

This notebook contains the baseline code for the training a vanilla [Mask RCNN](https://arxiv.org/abs/1703.06870) model for the [crowdAI Mapping Challenge](https://www.crowdai.org/challenges/mapping-challenge).

This code is adapted from the [Mask RCNN]() tensorflow implementation available here : [https://github.com/matterport/Mask_RCNN](https://github.com/matterport/Mask_RCNN).

First we begin by importing all the necessary dependencies : 

In [1]:
import os
import sys
import time
import numpy as np

# Download and install the Python COCO tools from https://github.com/waleedka/coco
# That's a fork from the original https://github.com/pdollar/coco with a bug
# fix for Python 3.
# I submitted a pull request https://github.com/cocodataset/cocoapi/pull/50
# If the PR is merged then use the original repo.
# Note: Edit PythonAPI/Makefile and replace "python" with "python3".
#  
# A quick one liner to install the library 
# !pip install git+https://github.com/waleedka/coco.git#subdirectory=PythonAPI

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

from mrcnn.evaluate import build_coco_results, evaluate_coco
from mrcnn.dataset import MappingChallengeDataset

import zipfile
import urllib.request
import shutil


## Dataset location 
Now we have to download all the files in the datasets section and untar them to have the following structure :
```
├── data
|   ├── pretrained_weights.h5 (already included in this repository)
│   ├── test
│   │   └── images/
│   │   └── annotation.json
│   ├── train
│   │   └── images/
│   │   └── annotation.json
│   └── val
│       └── images/
│       └── annotation.json
```
Note that the `pretrained_weights.h5` (available at [https://www.crowdai.org/challenges/mapping-challenge/dataset_files](https://www.crowdai.org/challenges/mapping-challenge/dataset_files)) are the weights used for the baseline submission, and are obtained by running the learning schedule mentioned later in the experiment. In the said experiment, the initial weights used can be found [here](https://github.com/matterport/Mask_RCNN/releases/download/v2.1/mask_rcnn_balloon.h5). 

In [2]:
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils


PRETRAINED_MODEL_PATH = os.path.join(ROOT_DIR,"data/" "pretrained_weights.h5")
LOGS_DIRECTORY = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


## Experiment Configuration

In [3]:
class MappingChallengeConfig(Config):
    """Configuration for training on data in MS COCO format.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "crowdai-mapping-challenge"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 5

    # Uncomment to train on 8 GPUs (default is 1)
    GPU_COUNT = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # 1 Backgroun + 1 Building

    STEPS_PER_EPOCH=90
    VALIDATION_STEPS=20

    IMAGE_MAX_DIM=320
    IMAGE_MIN_DIM=320
    
    MEAN_PIXEL = [81.16231469, 86.53528546, 64.72005973]

config = MappingChallengeConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     40
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      8
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 5
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  320
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  320
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [320 320   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_mask_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'rpn_class_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE            

## Instantiate Model

In [4]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=LOGS_DIRECTORY)
# Load pretrained weights
model_path = PRETRAINED_MODEL_PATH
model.load_weights(model_path, by_name=True)

Instructions for updating:
Colocations handled automatically by placer.


## Load Training and Validation Dataset

In [5]:
# Load training dataset
dataset_train = MappingChallengeDataset()
dataset_train.load_dataset(dataset_dir=os.path.join("data", "train"), load_small=False)
dataset_train.prepare()

# Load validation dataset
dataset_val = MappingChallengeDataset()
val_coco = dataset_val.load_dataset(dataset_dir=os.path.join("data", "val"), load_small=False, return_coco=True)
dataset_val.prepare()

Annotation Path  data/train/annotation.json
Image Dir  data/train/images
loading annotations into memory...
Done (t=2.93s)
creating index...
index created!
Annotation Path  data/val/annotation.json
Image Dir  data/val/images
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!


## Train

In [6]:
# *** This training schedule is an example. Update to your needs ***

# # Training - Stage 1
# print("Training network heads")
# model.train(dataset_train, dataset_val,
#             learning_rate=config.LEARNING_RATE,
#             epochs=10,
#             layers='heads')

# # Training - Stage 2
# # Finetune layers from ResNet stage 4 and up
# print("Fine tune Resnet stage 4 and up")
# model.train(dataset_train, dataset_val,
#             learning_rate=config.LEARNING_RATE,
#             epochs=10,
#             layers='4+')

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=20,
            layers='all')

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/yoninachmany/crowdai-mapping-challenge-mask-rcnn/logs/crowdai-mapping-challenge20190428T1455/mask_rcnn_crowdai-mapping-challenge_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed

/home/yoninachmany/.local/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yoninachmany/.local/lib/python3.5/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
90/90 [==============================] - 402s 4s/step - loss: 3.8146 - rpn_class_loss: 0.6806 - rpn_bbox_loss: 1.6607 - mrcnn_class_loss: 0.4054 - mrcnn_bbox_loss: 0.4848 - mrcnn_mask_loss: 0.5830 - val_loss: 2.5160 - val_rpn_class_loss: 0.2361 - val_rpn_bbox_loss: 1.0742 - val_mrcnn_class_loss: 0.2789 - val_mrcnn_bbox_loss: 0.4515 - val_mrcnn_mask_loss: 0.4754
Epoch 2/10
90/90 [==============================] - 102s 1s/step - loss: 2.1653 - rpn_class_loss: 0.1892 - rpn_bbox_loss: 0.9113 - mrcnn_class_loss: 0.2505 - mrcnn_bbox_loss: 0.3763 - mrcnn_mask_loss: 0.4380 - val_loss: 2.5233 - val_rpn_class_loss: 0.2466 - val_rpn_bbox_loss: 1.0661 - val_mrcnn_class_loss: 0.2758 - val_mrcnn_bbox_loss: 0.4524 - val_mrcnn_mask_loss: 0.4824
Epoch 3/10
90/90 [==============================] - 102s 1s/step - loss: 2.2847 - rpn_class_loss: 0.1891 - rpn_bbox_loss: 0.9751 - mrcnn_class_loss: 0.2774 - mrcnn_bbox_loss: 0.3927 - mrcnn_mask_loss: 0.4505 - val_loss: 2.5026 - val_rpn_class_loss: 0

82/90 [==========================>...] - ETA: 8s - loss: 2.2286 - rpn_class_loss: 0.1781 - rpn_bbox_loss: 0.9486 - mrcnn_class_loss: 0.2853 - mrcnn_bbox_loss: 0.3792 - mrcnn_mask_loss: 0.4374

85/90 [===========================>..] - ETA: 5s - loss: 2.2309 - rpn_class_loss: 0.1777 - rpn_bbox_loss: 0.9502 - mrcnn_class_loss: 0.2859 - mrcnn_bbox_loss: 0.3796 - mrcnn_mask_loss: 0.4375

...

48/90 [===============>..............] - ETA: 53s - loss: 1.9906 - rpn_class_loss: 0.1385 - rpn_bbox_loss: 0.7967 - mrcnn_class_loss: 0.2821 - mrcnn_bbox_loss: 0.3463 - mrcnn_mask_loss: 0.4270

50/90 [===============>..............] - ETA: 50s - loss: 1.9878 - rpn_class_loss: 0.1382 - rpn_bbox_loss: 0.7950 - mrcnn_class_loss: 0.2816 - mrcnn_bbox_loss: 0.3458 - mrcnn_mask_loss: 0.4272

53/90 [================>.............] - ETA: 46s - loss: 1.9825 - rpn_class_loss: 0.1377 - rpn_bbox_loss: 0.7921 - mrcnn_class_loss: 0.2806 - mrcnn_bbox_loss: 0.3449 - mrcnn_mask_loss: 0.4272

56/90 [=================>............] - ETA: 42s - loss: 1.9768 - rpn_class_loss: 0.1371 - rpn_bbox_loss: 0.7889 - mrcnn_class_loss: 0.2800 - mrcnn_bbox_loss: 0.3437 - mrcnn_mask_loss: 0.4270

90/90 [==============================] - 133s 1s/step - loss: 1.9454 - rpn_class_loss: 0.1364 - rpn_bbox_loss: 0.7707 - mrcnn_class_loss: 0.2751 - mrcnn_bbox_loss: 0.3368 - mrcnn_mask_loss: 0.4264 - val_loss: 2.4469 - val_rpn_class_loss: 0.2194 - val_rpn_bbox_loss: 1.0238 - val_mrcnn_class_loss: 0.3048 - val_mrcnn_bbox_loss: 0.4298 - val_mrcnn_mask_loss: 0.4690
Epoch 11/20


Now you can monitor the training by running : 
```
tensorboard --logdir=logs/[path-to-your-experiment-logdir]
```
and if everything works great, you should see something like : 
![loss-plot](images/loss-plot.png)

# Author
Sharada Mohanty [sharada.mohanty@epfl.ch](sharada.mohanty@epfl.ch)